In [1]:
# Instalar as bibliotecas necessárias (rode no colab)
!pip install -U google-genai -q
!pip install -q google-adk

# Importações e configuração
import os
from google.colab import userdata
import google.generativeai as genai  # Biblioteca oficial para Gemini (genai)

# Configuração dos tools disponíveis no modelo Gemini (só o nome da ferramenta)
TOOLS = ["google_search"]

# Quebrando o Texto por Linhas
from IPython.display import HTML, Markdown
import textwrap


# Configurar chave da API do Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


# Importações do ADK
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types # This import is for types, not the main model
from datetime import date
import ast  # Segurança ao avaliar dicionários do agente


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [2]:
#função auxiliar chamar agentes
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text:
                    final_response += part.text + "\n"
    return final_response

# MEMÓRIA GLOBAL (pode ser exportada para JSON, Planilha, etc.)
perfil_usuario = {}

In [3]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
# DICIONÁRIO
# melhorar: criar em um arquivo externo .py
metodos_produtividade = {
    "GTD": {
        "organizacao": ["metas", "livre"],
        "descricao": (
            "O método Getting Things Done (GTD) foca em capturar todas as tarefas e compromissos "
            "fora da mente, organizá-los em listas específicas e revisá-los regularmente para aumentar a produtividade."
        ),
        "exemplo_uso": (
            "Liste todas as suas tarefas, categorize-as por contexto (ex: 'telefonemas', 'compras'), "
            "e revise semanalmente para priorizar o que deve ser feito."
        ),
        "ferramentas": ["Todoist", "Evernote", "Notion"]
    },
    "Ivy Lee": {
        "organizacao": ["metas", "time_blocking"],
        "descricao": (
            "Método simples que prioriza até 6 tarefas importantes para o dia seguinte, "
            "executando-as em ordem, ajudando a focar no essencial."
        ),
        "exemplo_uso": (
            "No final do dia, escreva as 6 tarefas mais importantes para amanhã, "
            "comece pela primeira e só passe para a próxima quando terminar."
        ),
        "ferramentas": ["Bloco de notas", "Todoist"]
    },
    "Pomodoro": {
        "organizacao": ["time_blocking", "livre"],
        "descricao": (
            "Trabalhar em blocos de tempo focado de 25 minutos, seguidos por pausas curtas, "
            "aumentando a concentração e prevenindo fadiga."
        ),
        "exemplo_uso": (
            "Configure um timer para 25 minutos e trabalhe sem distrações. "
            "Após o alarme, faça uma pausa de 5 minutos. Repita 4 vezes e faça uma pausa maior."
        ),
        "ferramentas": ["Pomodone", "TomatoTimer", "Forest"]
    },
    "Kanban": {
        "organizacao": ["to_do", "livre"],
        "descricao": (
            "Sistema visual que organiza tarefas em colunas como 'A fazer', 'Fazendo' e 'Feito', "
            "facilitando o acompanhamento do fluxo de trabalho."
        ),
        "exemplo_uso": (
            "Crie cartões para cada tarefa e mova-os entre as colunas conforme o progresso, "
            "visualizando claramente o status das atividades."
        ),
        "ferramentas": ["Trello", "Asana", "ClickUp"]
    }
}

In [5]:
"""# Função de busca usando Gemini com Google Search tool
def buscar_com_google_gemini(query: str):
    # Cria o modelo Gemini com a ferramenta google_search ativada
    model = genai.GenerativeModel(
        model_name="gemini-1.5-pro-latest",
        tools=[{"name": "google_search"}]  # Apenas 'name' da ferramenta, sem 'type'
    )
    # Gera o conteúdo usando o prompt e ativando a função de busca
    response = model.generate_content(query)
    return response.text"""

def buscar_com_google_gemini(query: str):
  model = genai.GenerativeModel(
      model_name="gemini-1.5-pro-latest",
      tools=[{"name": "google_search"}]
  )

  # Limita tamanho do prompt a 500 caracteres (ajustável)
  query_limpa = query.strip()[:500]

  try:
      response = model.generate_content(query_limpa)
      return response.text
  except Exception as e:
      return f"Erro ao buscar: {e}"

In [6]:
# FUNÇÃO ENTREVISTA
def entrevista_interativa():
    respostas = {}

    perguntas = {
        "segmento": "1. Qual é o segmento principal da sua empresa? (Ex: Alimentação, Moda, Tecnologia, Serviços, etc.)",
        "tipo_negocio": "2. Seu tipo de negócio se encaixa mais como: e-commerce, venda de produtos físicos, prestação de serviços, consultoria, ou outro?",
        "objetivo": "3. Qual é o seu principal objetivo para o seu negócio neste momento? (Ex: Aumentar vendas, organizar minhas finanças, melhorar meu marketing, otimizar meu tempo, etc.)",
        "organizacao": "4. Pensando em como você geralmente lida com suas tarefas e seu tempo, qual das opções abaixo descreve melhor seu estilo de produtividade?\n\
a) Focado em metas\nb) Organizado por tempo\nc) Flexível e adaptável\nd) Orientado por listas"
    }
    for chave, pergunta in perguntas.items():
        resposta = input(pergunta + "\n> ")
        respostas[chave] = resposta

    return respostas

In [7]:
# AGENTE 1 - PERFILADOR
# melhorar: coletar informações durante qualquer interação (de forma dinamica) para adicionar novas variaveis dentro de perfil
def agente_classificador_de_perfil(respostas):
    """
    Interpreta e classifica as respostas da entrevista_interativa() com base em perfis padronizados.
    Retorna um dicionário com labels estruturados.
    """
    perfil = {}

    # Segmento (mantém texto original do usuário)
    segmento = respostas.get("segmento", "").strip()
    perfil["segmento"] = segmento

    #Tipo de negócio
    tipo = respostas.get("tipo_negocio", "").lower()
    # Categorias com palavras relacionadas
    palavras_produto = ["produto", "produtos", "fabricação", "fabricacao", "industrial"]
    palavras_servico = ["serviço", "servico", "prestação", "consultoria", "atendimento", "freelancer"]
    palavras_online = ["e-commerce", "loja virtual", "digital", "online", "marketplace"]
    palavras_fisico = ["físico", "fisico", "loja física", "presencial", "ponto de venda"]

    # Verifica a qual categoria pertence
    if any(p in tipo for p in palavras_produto):
        perfil["tipo_negocio"] = "produto"
    elif any(p in tipo for p in palavras_servico):
        perfil["tipo_negocio"] = "servico"
    elif any(p in tipo for p in palavras_online):
        perfil["tipo_negocio"] = "online"
    elif any(p in tipo for p in palavras_fisico):
        perfil["tipo_negocio"] = "fisico"
    else:
        perfil["tipo_negocio"] = "hibrido"

    # Objetivo
    objetivo = respostas.get("objetivo", "").lower()
    if "vender" in objetivo:
        perfil["objetivo"] = "vender_mais"
    elif "organizar" in objetivo or "rotina" in objetivo:
        perfil["objetivo"] = "organizar_rotina"
    elif "tempo" in objetivo or "automatizar" in objetivo:
        perfil["objetivo"] = "ganhar_tempo"
    else:
        perfil["objetivo"] = "neutro"

    # Estilo de organização
    org = respostas.get("organizacao", "").lower().strip()

    if org.startswith("b"):
        perfil["organizacao"] = "time_blocking"
    elif org.startswith("a"):
        perfil["organizacao"] = "metas"
    elif org.startswith("d"):
        perfil["organizacao"] = "to_do"
    elif org.startswith("c"):
        perfil["organizacao"] = "livre"
    else:
        perfil["organizacao"] = "indefinido"

    def adicionar_metodo(perfil, metodos_produtividade):
      # Seleciona os métodos baseado na organização
      organizacao = perfil.get("organizacao", "")
      metodos_para_organizacao = [
          metodo for metodo, dados in metodos_produtividade.items()
          if organizacao in dados["perfil_indicado"]
      ]
      # Atualiza o perfil com os métodos encontrados ou usa ["GTD"] como padrão
      perfil["metodos_produtividade"] = metodos_para_organizacao or ["GTD"]

    return perfil
    print (to_markdown(perfil))

In [8]:
# AGENTE 2 APRESENTADOR
def agente_apresentador_de_perfil(perfil_usuario):
    agente = Agent(
        name="agente_apresentador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente pessoal empático e inteligente.
        Sua missão é ler o perfil de um empreendedor e:

        1. Fazer um comentário breve, simpático e acolhedor sobre o estilo da pessoa.
        2. Mostrar que você compreende o contexto e o ramo de atuação dela.
        3. Reafirmar que todas as suas respostas a partir de agora serão adaptadas a esse perfil.
        4. Perguntar, com naturalidade e leveza, o que ela precisa resolver HOJE.

        Use linguagem clara, respeitosa e personalizada.
        Adapte o tom da resposta conforme o estilo de foco ou organização do usuário.
        Pode usar emojis se o perfil sugerir algo mais informal.
        Evite parecer robótico. Soe como alguém que entende gente de verdade.
        """,
        description="Apresentador de perfil e primeiro contato amigável com o usuário"
    )

    prompt = f"Perfil do usuário:\n{perfil_usuario}"
    print("🌟 Apresentando o perfil e iniciando o atendimento personalizado...\n")
    resposta = call_agent(agente, prompt)
    print(resposta)
    return to_markdown(resposta)

In [9]:
# AGENTE 3 EXECUTOR
def agente_executor_personalizado(pedido_usuario: str, perfil: dict) -> str:
    agente = Agent(
        name="agente_executor_personalizado",
        model="gemini-2.0-flash",
        # melhorar: data dinamica
        instruction=f""" Você é um assistente prático e visionário, feito para apoiar pequenos empreendedores brasileiros com base nas tendências mais recentes (até maio de 2025)
         e orientações do Sebrae. Use um tom encorajador e, quando apropriado, um toque de humor leve para motivar.

        → Baseie suas respostas no perfil do usuário:
          - Organização: {perfil.get('organizacao')}
          - Tipo de negócio: {perfil.get('tipo_negocio')}
          - Segmento: {perfil.get('segmento')}
          - Objetivo principal: {perfil.get('objetivo')}
          - Metodos de produtividade recomendados: {perfil.get('metodos_produtividade')}

        → Adapte sua comunicação com base nesses perfis. Use uma tabela interna para modular tom, formato e sugestões.

        → Sempre que o tema permitir, pergunte sutilmente sobre ações sustentáveis no negócio. Enfatize que empresas comprometidas com o planeta tendem a liderar o futuro do mercado.

        → Seja sempre:
          - Claro e conciso (respostas curtas, sem floreios)
          - Personalizado (nada genérico)
          - Focado em ação prática, metas e acompanhamento
          - Atualizado com dados do Sebrae, tendências digitais e comportamento do consumidor

        → Incentive:
          - Participação em conselhos e redes de apoio locais
          - Cursos e capacitações
          - Inovação constante
          - O associativismo como caminho de crescimento

       → Se o pedido estiver incompleto ou muito geral, utilize seu conhecimento sobre o perfil e incentive o usuário a dar mais contexto com leveza — mas continue ajudando com o que for possível.
      → Se o metodo de produtividade do usuário for conhecido, você pode sugerir um método que combine com ele e com a solicitação que ele te fez. Use linguagem leve, como: Nesse caso, talvez o método X funcione bem pra você.
      Pedido do usuário: "{pedido_usuario}"
        """,
        description="Executor Personalizado: adapta respostas e sugestões ao perfil do empreendedor"
    )

    resposta = call_agent(agente, pedido_usuario)
    return to_markdown(resposta)

In [ ]:
# Bloco inicial: apresentação e decisão
ola = f"""
      👋 Olá! Sou o Empreend.AI, seu novo sócio digital! Para te ajudar de forma mais personalizada,
      quero te fazer 4 perguntas essenciais sobre sua empresa e seu estilo de trabalho. Podemos começar?
      Ou prefere ir direto ao ponto e me dizer em que posso te ajudar HOJE?

      """
display(to_markdown(ola))

resposta_inicial = input("✏️ Digite 'começar' ou 'direto':\n> ").lower()

if resposta_inicial == "direto":
    perfil_usuario = {"modo_direto": True}
    print("👍 Perfeito! Pode me dizer com o que você precisa de ajuda agora.")
else:
    respostas_usuario = entrevista_interativa()
    perfil_usuario = agente_classificador_de_perfil(respostas_usuario)
    agente_apresentador_de_perfil(perfil_usuario)

# Loop contínuo da conversa
print("\n🚀 Agora você pode fazer seus pedidos. Digite 'sair' para encerrar.\n")
comandos_sair = {"sair", "exit", "quit", "parar", "encerrar", "fim", "tchau"}

while True:
    pedido = input("> ").strip()
    if pedido.lower() in comandos_sair:
        print("Até logo! Foi um prazer ajudar você. 🌟")
        break

    # Dispara busca se a frase for maior que 4 palavras (para evitar buscas inúteis)
    if len(pedido.split()) > 4:
        busca = buscar_com_google_gemini(pedido)
        prompt_completo = f"{pedido}\n\nResultados da busca atual:\n{busca}"
    else:
        prompt_completo = pedido

    # Aqui você pode chamar sua função/agent personalizada que responde baseado no prompt completo
    # Exemplo fictício, substitua pela sua implementação:
    resposta = agente_executor_personalizado(prompt_completo, perfil_usuario)

    display(resposta)

"""while True:
    pedido = input("> ").strip()
    if pedido.lower() in comandos_sair:
        print("Até logo! Foi um prazer ajudar você. 🌟")
        break

    # Busca SEMPRE que o pedido tiver conteúdo relevante
    if len(pedido.split()) > 4:  # Evita busca para mensagens muito curtas
        busca = buscar_com_google_gemini(pedido)
        prompt_completo = f"{pedido}\n\nResultados da busca atual:\n{busca}"
    else:
        prompt_completo = pedido

    resposta = agente_executor_personalizado(prompt_completo, perfil_usuario)
    display(resposta)"""

> 
>       👋 Olá! Sou o Empreend.AI, seu novo sócio digital! Para te ajudar de forma mais personalizada,
>       quero te fazer 4 perguntas essenciais sobre sua empresa e seu estilo de trabalho. Podemos começar?
>       Ou prefere ir direto ao ponto e me dizer em que posso te ajudar HOJE?
> 
>       